In [1]:
df = spark \
        .read.format('csv')\
        .option('header', 'true')\
        .load('hdfs:///user/gpfl/beer/beer_reviews.csv')

In [2]:
df.createOrReplaceTempView('beer')

In [4]:
df_beer = spark.sql('select * from beer')

In [6]:
df_beer.count(), len(df_beer.columns)

(1586614, 13)

In [7]:
cols = df_beer.columns
n_cols = len(cols)

for i in range(1, n_cols, 5):
    if i - 5 < n_cols:
        df_beer.select(cols[i:i+5]).show(5)
    else:
        df_beer.select(cols[i, n_cols]).show(5)
    

+--------------------+-----------+--------------+------------+-----------------+
|        brewery_name|review_time|review_overall|review_aroma|review_appearance|
+--------------------+-----------+--------------+------------+-----------------+
|     Vecchio Birraio| 1234817823|           1.5|           2|              2.5|
|     Vecchio Birraio| 1235915097|             3|         2.5|                3|
|     Vecchio Birraio| 1235916604|             3|         2.5|                3|
|     Vecchio Birraio| 1234725145|             3|           3|              3.5|
|Caldera Brewing C...| 1293735206|             4|         4.5|                4|
+--------------------+-----------+--------------+------------+-----------------+
only showing top 5 rows

+------------------+--------------------+-------------+------------+--------------------+
|review_profilename|          beer_style|review_palate|review_taste|           beer_name|
+------------------+--------------------+-------------+-----------

## 1. Estilo de Cerveja mais bem avaliada

In [18]:
import pyspark.sql.functions as f

df_beer\
    .groupBy('beer_style')\
    .agg(f.mean('review_palate').alias('review_palate'),
          f.mean('review_taste').alias('review_taste'))\
    .sort('review_taste', ascending=False)\
    .show(truncate=False)

+--------------------------------+------------------+------------------+
|beer_style                      |review_palate     |review_taste      |
+--------------------------------+------------------+------------------+
|Eisbock                         |4.113593691325573 |4.211603454750281 |
|Quadrupel (Quad)                |4.124986177153599 |4.210908990379299 |
|American Double / Imperial Stout|4.0986687703382305|4.187230056207475 |
|American Wild Ale               |4.040631673597842 |4.149938181409464 |
|Russian Imperial Stout          |4.086921982671027 |4.1495686231040665|
|Gueuze                          |4.046679980029955 |4.127142619404227 |
|Lambic - Unblended              |3.964991023339318 |4.124326750448833 |
|American Double / Imperial IPA  |4.023128278493085 |4.091280226106982 |
|Flanders Red Ale                |3.970888355342137 |4.090636254501801 |
|Weizenbock                      |3.9907033574160646|4.077348066298343 |
|Old Ale                         |3.945045228864857

## 2. Cerveja mais bem avaliada

In [38]:
df_beer.groupby('beer_name')\
       .agg(f.mean('review_overall').alias('review_overall'))\
       .sort('review_overall', ascending=False)\
       .show(10, truncate=False)

+---------------------------------+--------------+
|beer_name                        |review_overall|
+---------------------------------+--------------+
|Lucky Malt                       |5.0           |
|Spruced Up Wheat                 |5.0           |
|961 IPA (India Pale Ale)         |5.0           |
|Snail Trail Pale                 |5.0           |
|Black Opal IPA                   |5.0           |
|Edsten Double Plus               |5.0           |
|Nine Tailed Fox                  |5.0           |
|Saint Slater Irish Stout         |5.0           |
|3 Threads                        |5.0           |
|Double Bubble DIPA With Bubblegum|5.0           |
+---------------------------------+--------------+
only showing top 10 rows



## 3. Cervejaria melhor avaliada

In [34]:
df_beer.groupby('brewery_name')\
       .agg(f.mean('review_taste').alias('review_taste'))\
       .sort('review_taste', ascending=False)\
       .show(10, truncate=False)

+-----------------------------+------------+
|brewery_name                 |review_taste|
+-----------------------------+------------+
|Hakone Beer                  |5.0         |
|Elizabeth Street Brewery     |5.0         |
|Thai Me Up                   |5.0         |
|Binghams Brewery             |5.0         |
|Rascal Creek Brewing Co.     |5.0         |
|Ludwig Roth Bierbrauerei GmbH|5.0         |
|The Cellar Grill             |5.0         |
|Sherlock's Home              |4.9         |
|Edsten Brewing Company       |4.75        |
|Sair Inn                     |4.75        |
+-----------------------------+------------+
only showing top 10 rows

